In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
from scipy import interpolate
pi = np.pi
np.set_printoptions(precision=3)




In [2]:
def createPeriodicSplines( X, Y ):
    tck, u = interpolate.splprep([X, Y], s=0, per=1)
    return tck

def createSplines( X, Y ):
    tck, u = interpolate.splprep([X, Y], s=0)
    return tck

def computeNodeLocations( N, tck ):
    t = np.linspace( 0, 1, N )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    x, y = interpolate.splev(t, tck)
    return x, y

def computeNormParameterizationDerivative( N, tck ):
    t = np.linspace( 0, 1, N )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    dx_dt, dy_dt = interpolate.splev(t, tck, der=1)
    ds_dt = np.sqrt( dx_dt**2 + dy_dt**2 )
    return ds_dt

def computeRadial( N, tck ):
    t = np.linspace( 0, 1, N )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    rx, ry = interpolate.splev(t, tck)
    norm = np.sqrt( rx**2 + ry**2 )
    return rx/norm, ry/norm

def computeTangent( N, tck ):
    t = np.linspace( 0, 1, N )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    tx, ty = interpolate.splev(t, tck, der=1)
    norm = np.sqrt( tx**2 + ty**2 )
    return tx/norm, ty/norm

def computeNormal( N, tck ):
    t = np.linspace( 0, 1, N )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    nx, ny = interpolate.splev(t, tck, der=2)
    norm = -np.sqrt( nx**2 + ny**2 )
    return nx/norm, ny/norm

def computeBinormal( N, tck ):
    tx, ty = computeTangent( N, tck )
    nx, ny = computeNormal( N, tck )
    tangentVector = np.vstack( (tx, ty) )
    normalVector = np.vstack( (nx, ny) )
    binormal = np.cross( np.transpose(tangentVector), np.transpose(normalVector) )
    return -binormal

def computeChordLength( X ):
    return np.max(X) - np.min(X)

def createWakeGeometry( X, Y, AOA ):
    chordLength = computeChordLength( X )
    wakeLength = 5 * chordLength
    x0 = .5 * ( X[0] + X[-1] )
    y0 = .5 * ( Y[0] + Y[-1] )
    xf = x0 + wakeLength * np.cos( AOA * pi/180 )
    yf = y0 + wakeLength * np.sin( AOA * pi/180 )
    Xw = np.asarray( [x0, xf] )
    Yw = np.asarray( [y0, yf] )
    tck, u = interpolate.splprep([Xw, Yw], k=1, s=0, quiet=True )
    return tck

def computeWakeNormal( Nw, AOA ):
    nxw = -np.sin( AOA * pi/180 ) * np.ones( Nw )
    nyw = np.cos( AOA * pi/180 ) * np.ones( Nw )
    return nxw, nyw
    




In [20]:
def computeGreenNormal2D( x, y, xj, yj , normalx, normaly ):
    rj2 = (x - xj)**2 + (y - yj)**2
    dGdr = ( 1 / 2 / pi ) / rj2 
    gradrn = normalx * (x - xj) + normaly * (y - yj)
    return -dGdr * gradrn

def assembleMatrixSystem( X, Y, N, Nw, U0, AOA ):
    ## check if the geometry has sharp trailing edge
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        tck = createSplines( X, Y )
    x, y = computeNodeLocations( N+1, tck )
    theta = np.arctan2( y[:], x[:] )
    ds_dt = computeNormParameterizationDerivative( N+1, tck )
    nx, ny = computeNormal( N+1, tck )
    binormal = computeBinormal( N+1, tck )
    c = computeChordLength( X )

    ## correct normal direction
    nx = binormal * nx
    ny = binormal * ny
    
    ## generate system of equations
    A = np.identity( N+1 )
    b = np.zeros( N+1 )
    t = np.linspace( 0, 1, N+1 )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    
    dt = t[1:] - t[:-1]
    tolo = 1
    ## matrix influence due to airfoil
    for i in range( N+1 ):
        b[i] = U0 * ( x[i] * np.cos( AOA * pi / 180 ) +  y[i] * np.sin( AOA * pi / 180 ) )
        for j in range( N ):
            ## first comp
            if i != j and not( i == 0 and j == N-1):
                wj = computeGreenNormal2D( x[i], y[i], x[j], y[j], nx[j], ny[j] ) * ds_dt[j] * dt[j]/2
                if np.abs(wj) > tolo:
                    print( i, j )
                    wj = 0
                A[i, i] = A[i, i] - wj
                A[i, j] = A[i, j] + wj
            
            ## second comp
            if i != j+1 and not( i == N+1 and j == 0): 
                wj = computeGreenNormal2D( x[i], y[i], x[j+1], y[j+1], nx[j+1], ny[j+1] ) * ds_dt[j+1] * dt[j]/2
                if np.abs(wj) > tolo:
                    print( i, j )
                    wj = 0
                A[i, i] = A[i, i] - wj
                A[i, j+1] = A[i, j+1] + wj
            
    ## matrix influence due to wake
    # create wake representation
    tckw = createWakeGeometry( X, Y, AOA )
    xw, yw = computeNodeLocations( Nw+1, tckw )
    nxw, nyw = computeWakeNormal( Nw+1, AOA )
    ds_dtw = computeNormParameterizationDerivative( Nw+1, tckw )
#     ds = 5 * c / Nw
    t = np.linspace( 0, 1, Nw+1 )
    t = np.split( t, 2 )
    t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
    t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
    t = np.concatenate( ( t[0], t[1] ) )
    
    dtw = t[1:] - t[:-1]
    
    for i in range( N+1 ):
        for k in range( Nw ):
            ## first part
            aik = computeGreenNormal2D( x[i], y[i], xw[k], yw[k], nxw[k], nyw[k] ) * ds_dtw[k]*dtw[k] / 2
            if np.abs( aik ) > tolo:
#                 print( i, j )
                aik = 0
            A[i, 0] = A[i, 0] + aik/4 
            A[i, 1] = A[i, 1] - aik/2
            A[i, 2] = A[i, 2] + aik/4
            
            A[i, N] = A[i, N] - aik
            ## second part
            aik = computeGreenNormal2D( x[i], y[i], xw[k+1], yw[k+1], nxw[k+1], nyw[k+1] ) * ds_dtw[k+1]*dtw[k] / 2
            if np.abs( aik ) > tolo:
#                 print( i, j )
                aik = 0
            A[i, 0] = A[i, 0] + aik
            A[i, N] = A[i, N] - aik
    kuttaRow = np.zeros( N+1 )
#     kuttaRow[0] = -3/2
#     kuttaRow[1] = 2
#     kuttaRow[2] = -1/2
#     kuttaRow[-1] = 3/2
#     kuttaRow[-2] = -2
#     kuttaRow[-3] = 1/2
    
    kuttaRow[0] = -1
    kuttaRow[1] = 1
#     kuttaRow[2] = -1/2
    kuttaRow[-1] = 1
    kuttaRow[-2] = -1
#     kuttaRow[-3] = 1/2
    
    A[N, :] = kuttaRow
    b[N] = 0
   
    return x, y, theta, A, b, ds_dt


        









In [24]:
M = 1024 + 1
N = 1024 + 1
Nw = 33
AOA = 5
U0 = 1

## create geometry
c = 1
e = 0.5
a = c * ( 1 + e ) / 4

t = np.linspace( 0, 1, M )
X0 = - e * c / 4
X = a * np.cos( 2 * pi * t ) + X0
Y = a * np.sin( 2 * pi * t )
Z = X + 1j*Y
Z = Z + c**2/Z/16
X = np.real(Z)
Y = np.imag(Z)



## create system of equations
x, y, theta, A, b, ds_dt = assembleMatrixSystem( X, Y, N, Nw, U0, AOA )

# ## solve system of equations
# phi = np.linalg.solve( A, b )
# print( A ) 

# ## plot solution
# fig, ax = plt.subplots(figsize=(8, 8))
# plt.grid(b=True, which='major', linestyle='--', linewidth=.5)
# plt.grid(b=True, which='minor', linestyle='--', linewidth=.1)
# plt.minorticks_on()
# plt.xlabel('x')
# ax.plot( np.concatenate( (x,[x[0]]) ), np.concatenate( (y,[y[0]]) ), '--' )

# ax.plot( x, phi, '.',label="Numerical Solution" )
# # ax.plot( np.real(z), np.real(w), '--', label="Theoretical Solution" )

# ## compute pressure coefficient

# t = np.linspace( 0, 1, N+1 )
# t = np.split( t, 2 )
# t[0] = -np.cos( t[0] * pi*2 )/4 + 1/4
# t[1] = np.cos( t[1] * pi*2 )/4 + 3/4
# t = np.concatenate( ( t[0], t[1] ) )
    
# dphi_ds = np.diff( phi ) / np.diff( t ) / ds_dt[1:]
# cp = 1 - (dphi_ds/U0)**2
# # ax.plot( x[:-1], cp, '--', label="Pressure Coefficient" ) 
# ax.legend()







0 1024
1 1023
1 1024
2 1022
2 1023
3 1021
3 1022
4 1020
4 1021
5 1019
5 1020
6 1018
1019 6
1020 4
1020 5
1021 3
1021 4
1022 2
1022 3
1023 1
1023 2
1024 0
1024 1
1025 0


/Users/daniellam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/daniellam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


###### 